In [1]:
import requests 
import pandas as pd
import os
from io import StringIO  # Import StringIO from the io module
import time  # Import the time module
import requests 
from urllib.parse import quote  # To URL encode SMILES
import plotly.express as px

Befor runing this code: 
1) create a .csv file containing a 'species_header' like 'species', 'names', etc. this column should contain the name of the species (or genuses by default) resolved by Open Tree of Life.
2) creta an empty folder (input_folder) and put the .csv file inside! 

That's all

DEFINE THE FOLLOWING PARAMETERS

In [2]:
#Paths
# Input folder containing CSV files
input_folder = '/mnt/c/Users/quirosgu/Desktop/MetaUV/'

SMILES_header = 'SMILES'

# Output folder for the processed CSV files
output_folder = '/mnt/c/Users/quirosgu/Desktop/test/output_data/'

FIRST RECOVER THE Q CODES FROM WIKIDATA AND UPDATE THE METADATA FILE

In [3]:
import pandas as pd
from io import StringIO
import requests
import time
import os
from rdkit import Chem

def process_csv_file(input_file, output_folder):
    # Read the original CSV file containing the SMILES
    df_compounds = pd.read_csv(input_file, encoding='ISO-8859-1')
    
    endpoint_url = "https://query.wikidata.org/sparql"
    q_codes = []
    placeholder_value = "Not Found"
    requests_per_minute = 10

    for index, row in df_compounds.iterrows():
        compound_smiles = row['SMILES']  # Assuming 'SMILES' is the column header for the compounds
        
        if isinstance(compound_smiles, str):
            # Convert SMILES to canonical form
            mol = Chem.MolFromSmiles(compound_smiles)
            if mol is not None:
                canonical_smiles = Chem.MolToSmiles(mol, isomericSmiles=True)

            #sparql
            sparql_query = f"""
            SELECT ?compound ?compoundLabel WHERE {{
              ?compound ?propertyForSMILES "{compound_smiles}".
              SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
            }}
            """
    
            response = requests.get(endpoint_url, headers={'Accept': 'text/csv'}, params={'query': sparql_query})
    
            if response.status_code == 200:
                csv_data = response.text
                df_qcode = pd.read_csv(StringIO(csv_data))
    
                if not df_qcode.empty:
                    q_code = df_qcode['compound'].values[0]  # Extract the Q code
                    q_codes.append(q_code)
                    print(f"Retrieved Q code {q_code} for compound with SMILES: {compound_smiles}")
                else:
                    q_codes.append(placeholder_value)
                    print(f"No Q code found for compound with SMILES: {compound_smiles}")
            else:
                q_codes.append(placeholder_value)
                print(f"Failed to retrieve Q code for compound with SMILES: {compound_smiles}. Status code: {response.status_code}")
    
            time.sleep(60 / requests_per_minute)
        else:
            q_codes.append(placeholder_value)
    
    df_compounds['wikidata_Qcode'] = q_codes
    output_csv_filename = os.path.join(output_folder, os.path.basename(input_file))
    df_compounds.to_csv(output_csv_filename, index=False)
    print(f"Saved updated CSV with Q codes to {output_csv_filename}")

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# List all CSV files in the input folder and process each one
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        input_file = os.path.join(input_folder, filename)
        process_csv_file(input_file, input_folder)

#once finished... check the lines for 'NOT FOUND' manually

ModuleNotFoundError: No module named 'rdkit'

NOW, LETS USE THE QCODES TO RECOVER ALL THE COMPOUNDS (WITH REFERENCES) PRESENT IN EACH QCODE AND SAVE THEM AS INDIVIDUAL CSV FILES

In [109]:
# Define your SPARQL query template
sparql_template = """
SELECT ?chemical_compound ?chemical_compoundLabel ?smiles_canonical ?mf ?mass ?chembl ?chemblLabel ?queried_taxa ?queried_taxaLabel ?reference ?referenceLabel
WHERE {
  ?chemical_compound wdt:P235 ?inchikey.
  OPTIONAL { ?chemical_compound wdt:P231 ?cas. }
  OPTIONAL { ?chemical_compound wdt:P233 ?smiles_canonical. }
  OPTIONAL { ?chemical_compound wdt:P234 ?inchi. }
  OPTIONAL { ?chemical_compound wdt:P592 ?chembl. }
  OPTIONAL { ?chemical_compound wdt:P662 ?pubchem. }
  OPTIONAL { ?chemical_compound wdt:P683 ?chebi. }
  OPTIONAL { ?chemical_compound wdt:P2017 ?smiles_isomeric. }
  OPTIONAL { ?chemical_compound wdt:P274 ?mf. }
  OPTIONAL { ?chemical_compound wdt:P2067 ?mass. }
  VALUES ?queried_taxa {
    wd:%s
  }
  {
    ?chemical_compound p:P703 ?stmt.
    ?queried_taxall (wdt:P171*) ?queried_taxa.
    ?stmt ps:P703 ?queried_taxall.
  }
  OPTIONAL {
    ?stmt prov:wasDerivedFrom ?ref.
    ?ref pr:P248 ?reference.
  }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""
# Define the Wikidata Query Service endpoint URL
endpoint_url = "https://query.wikidata.org/sparql"

# Define the number of requests allowed per minute (adjust according to Wikidata's rate limits)
requests_per_minute = 40

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Iterate through all CSV files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        input_file = os.path.join(input_folder, filename)

        # Load the CSV file with Q codes
        q_codes_df = pd.read_csv(input_file)

        # Iterate through the Q codes and run the SPARQL query with rate limiting
        for q_code in q_codes_df['wikidata_Qcode']:
            # Construct the specific query with the Q code
            sparql_query = sparql_template % q_code

            # Query Wikidata for chemical compounds and save the results
            response = requests.get(endpoint_url, headers={'Accept': 'text/csv'}, params={'query': sparql_query})

            if response.status_code == 200:
                csv_data = response.text
                df = pd.read_csv(StringIO(csv_data))

                # Group and aggregate the data for the specific Q code as described in the previous answer
                grouped_df = df.groupby("chemical_compound").agg({
                    "chemical_compoundLabel": "first",
                    "smiles_canonical": "first",
                    "mf": "first",
                    "mass": "first",
                    "chembl": "first",
                    "chemblLabel": "first",
                    "queried_taxa": "first",
                    "queried_taxaLabel": "first",
                    "reference": "|".join,
                    "referenceLabel": "|".join
                }).reset_index()

                # Save the grouped data as a CSV file with the Q code as the filename
                output_filename = os.path.join(output_folder, f"{q_code}.tsv")
                grouped_df.to_csv(output_filename, index=False, sep='\t')  # Use tab as the separator

                print(f"Saved grouped data for Q code {q_code} to {output_filename}")
            else:
                print(f"Failed to retrieve data for Q code {q_code}. Status code: {response.status_code}")

            # Implement rate limiting by waiting between requests
            time.sleep(60 / requests_per_minute)  # Wait for one minute divided by the desired requests per minute


Saved grouped data for Q code Q15287263 to /mnt/c/Users/quirosgu/Desktop/Inmuno/output_data/Q15287263.tsv
Saved grouped data for Q code Q8186515 to /mnt/c/Users/quirosgu/Desktop/Inmuno/output_data/Q8186515.tsv
Saved grouped data for Q code Q1882542 to /mnt/c/Users/quirosgu/Desktop/Inmuno/output_data/Q1882542.tsv
Saved grouped data for Q code Q4676527 to /mnt/c/Users/quirosgu/Desktop/Inmuno/output_data/Q4676527.tsv
Saved grouped data for Q code Q5657528 to /mnt/c/Users/quirosgu/Desktop/Inmuno/output_data/Q5657528.tsv
Saved grouped data for Q code Q11079071 to /mnt/c/Users/quirosgu/Desktop/Inmuno/output_data/Q11079071.tsv
Saved grouped data for Q code Q104912462 to /mnt/c/Users/quirosgu/Desktop/Inmuno/output_data/Q104912462.tsv
Saved grouped data for Q code Q4692127 to /mnt/c/Users/quirosgu/Desktop/Inmuno/output_data/Q4692127.tsv
Saved grouped data for Q code Q1949712 to /mnt/c/Users/quirosgu/Desktop/Inmuno/output_data/Q1949712.tsv
Saved grouped data for Q code Q159717 to /mnt/c/Users/qu

NOW LETS UPDATE THE CHEMICAL CLASSES OF THE REPORTED COMPOUNDS IN ALL THE FILES

In [111]:
# Define the NPClassifier API URL
npclassifier_url = "https://npclassifier.gnps2.org/classify"

# Define the maximum requests per minute
max_requests_per_minute = 40  # Adjust as needed

# Iterate through CSV files in the folder
for filename in os.listdir(output_folder):
    if filename.endswith(".tsv"):
        # Load the CSV file
        file_path = os.path.join(output_folder, filename)
        df_compounds = pd.read_csv(file_path, sep='\t', on_bad_lines='skip')

        # Drop rows with missing 'smiles_canonical'
        df_compounds.dropna(subset=['smiles_canonical'], inplace=True)

        # Initialize empty lists to store the predicted chemical classes
        pathways = []
        superclasses = []
        classes = []

        # Iterate through the compounds in the CSV
        for index, row in df_compounds.iterrows():
            smiles = row['smiles_canonical']

            # URL-encode the SMILES string
            encoded_smiles = quote(smiles)

            # Make an HTTP GET request to the NPClassifier API
            url = f"{npclassifier_url}?smiles={encoded_smiles}"
            response = requests.get(url)

            if response.status_code == 200:
                try:
                    # Parse the response to extract the chemical class predictions
                    result = response.json()
                    pathway = result.get("pathway_results", ["Not Classified"])[0]
                    superclass = result.get("superclass_results", ["Not Classified"])[0]
                    chemical_class = result.get("class_results", ["Not Classified"])[0]
                except Exception as e:
                    # Handle variations in the JSON structure
                    print(f"Error processing JSON for SMILES: {smiles}")
                    pathway = "Not Classified"
                    superclass = "Not Classified"
                    chemical_class = "Not Classified"

                pathways.append(pathway)
                superclasses.append(superclass)
                classes.append(chemical_class)
            else:
                # Handle API errors by adding 'API Error' as a placeholder
                print(f"API Error for SMILES: {smiles}")
                pathways.append("API Error")
                superclasses.append("API Error")
                classes.append("API Error")

            # Implement rate limiting by waiting between requests
            time.sleep(60 / max_requests_per_minute)

        # Add the predicted chemical class values to new columns in the DataFrame
        df_compounds['predicted_pathway'] = pathways
        df_compounds['predicted_superclass'] = superclasses
        df_compounds['predicted_class'] = classes
        
        # Add triples and pairs 
        
        # Convert non-string values to strings in columns A, B, and C
        df_compounds['predicted_pathway'] = df_compounds['predicted_pathway'].astype(str)
        df_compounds['predicted_superclass'] = df_compounds['predicted_superclass'].astype(str)
        df_compounds['predicted_class'] = df_compounds['predicted_class'].astype(str)

        #df_compounds['chemical_class'] =  df_compounds['predicted_pathway'] + '-' +  df_compounds['predicted_superclass'] + '-' +  df_compounds['predicted_class']
        df_compounds['chemical_superclass'] =  df_compounds['predicted_pathway'] + '-' +  df_compounds['predicted_superclass']

        # Save the updated CSV file with the same name
        updated_file_path = os.path.join(output_folder, filename)
        df_compounds.to_csv(updated_file_path, index=False, sep='\t')
        print(f"Saved updated CSV with predictions to {updated_file_path}")


Saved updated CSV with predictions to /mnt/c/Users/quirosgu/Desktop/Inmuno/output_data/Q10280832.tsv
Saved updated CSV with predictions to /mnt/c/Users/quirosgu/Desktop/Inmuno/output_data/Q10299053.tsv
Saved updated CSV with predictions to /mnt/c/Users/quirosgu/Desktop/Inmuno/output_data/Q10306429.tsv
Saved updated CSV with predictions to /mnt/c/Users/quirosgu/Desktop/Inmuno/output_data/Q10352100.tsv
Saved updated CSV with predictions to /mnt/c/Users/quirosgu/Desktop/Inmuno/output_data/Q10395363.tsv
Saved updated CSV with predictions to /mnt/c/Users/quirosgu/Desktop/Inmuno/output_data/Q1043450.tsv
Error processing JSON for SMILES: O=C(C1=CC=2C=CC(OC2C=C1O)(OC)C)C
Saved updated CSV with predictions to /mnt/c/Users/quirosgu/Desktop/Inmuno/output_data/Q104912462.tsv
Saved updated CSV with predictions to /mnt/c/Users/quirosgu/Desktop/Inmuno/output_data/Q10890773.tsv
Saved updated CSV with predictions to /mnt/c/Users/quirosgu/Desktop/Inmuno/output_data/Q110666042.tsv
Saved updated CSV with 

RECOVER THE FRECUENCE BY CHEMICAL CLASS AND ADDED TO THE GENERAL TABLE

In [112]:
# Step 1: Initialize variables to store data
total_compounds = {}
chemical_classes = {}
chemical_superclasses ={}
references = {}
hyperlinks = {}

# Step 2: Iterate through .tsv files in the folder
for filename in os.listdir(output_folder):
    if filename.endswith(".tsv"):
        qcode = filename.split(".")[0]  # Extract Qcode from the filename

        # Load the .tsv file into a DataFrame
        df_compounds = pd.read_csv(os.path.join(output_folder, filename), sep='\t')

        # Step 3: Calculate frequencies of chemical classes for each Qcode (excluding 'not classified')
        filtered_classes = df_compounds[df_compounds['predicted_class'] != 'Not Classified'] # filter the rows with unclassified compounds
        grouped = filtered_classes['predicted_class'].value_counts().reset_index()
        frequencies = grouped.apply(lambda row: f"{row['index']} {row['predicted_class']}", axis=1)
        chemical_classes[qcode] = "|".join(frequencies)
        
        # Step 3: Calculate frequencies of chemical superclasses for each Qcode (excluding 'not classified')
        filtered_sclasses = df_compounds[df_compounds['predicted_superclass'] != 'Not Classified'] # filter the rows with unclassified compounds
        sgrouped = filtered_sclasses['predicted_superclass'].value_counts().reset_index()
        frequencies = sgrouped.apply(lambda row: f"{row['index']} {row['predicted_superclass']}", axis=1)
        chemical_superclasses[qcode] = "|".join(frequencies)
        
        # Step 4: Combine references and hyperlinks
        references[qcode] = "|".join(df_compounds['reference'].dropna())
        hyperlinks[qcode] = "|".join(df_compounds['referenceLabel'].dropna())

        # Step 5: Calculate the total number of reported compounds
        total_compounds[qcode] = len(df_compounds)

# Step 6: Load the general info .csv into a DataFrame
csv_file = None
for filename in os.listdir(input_folder):
    if filename.endswith(".csv"):
        csv_file = os.path.join(input_folder, filename)
        break  # Stop searching after finding the first CSV file

if csv_file is not None:
    # Load the CSV file into the df_general_info DataFrame
    df_general_info = pd.read_csv(csv_file)

# Step 7: Create new columns for combined information
df_general_info['predicted_class'] = df_general_info['wikidata_Qcode'].map(chemical_classes)
df_general_info['predicted_superclass'] = df_general_info['wikidata_Qcode'].map(chemical_superclasses)
df_general_info['references'] = df_general_info['wikidata_Qcode'].map(references)
df_general_info['referenceLabel'] = df_general_info['wikidata_Qcode'].map(hyperlinks)

# Step 8: Create a new column for total compounds
df_general_info['total_compounds'] = df_general_info['wikidata_Qcode'].map(total_compounds)

# Step 7: Save the updated DataFrame to the .csv file

# Replace values matching the pattern with an empty cell
df_general_info['predicted_class'] = df_general_info['predicted_class'].replace('index|predicted_class', '')
df_general_info['predicted_superclass'] = df_general_info['predicted_superclass'].replace('index|predicted_superclass', '')
output_csv_file = input_folder + 'Full_results.csv'
df_general_info.to_csv(output_csv_file, index=False, sep =',')

Generate une plot by species (samples) and the chemical class

In [144]:
import pandas as pd
import plotly.express as px
import os

# Define the Vivid color palette (you can customize this as needed)
vivid_palette = px.colors.qualitative.Vivid

# Step 1: Initialize an empty DataFrame to store all data
all_data = pd.DataFrame(columns=['queried_taxaLabel', 'chemical_superclass'])

# Step 2: Read data from all .tsv files in the output_folder
for filename in os.listdir(output_folder):
    if filename.endswith(".tsv"):
        file_data = pd.read_csv(os.path.join(output_folder, filename), sep='\t')
        all_data = pd.concat([all_data, file_data])

# Step 3: Rename the "queried_taxaLabel" column to "species"
all_data.rename(columns={'queried_taxaLabel': 'species'}, inplace=True)

# Remove 'API Error-API Error' and 'Not Classified-Not Classified'
all_data = all_data[(all_data['chemical_superclass'] != 'API Error-API Error') & (all_data['chemical_superclass'] != 'Not Classified-Not Classified')]

# Step 4: Group and aggregate data to calculate recurrence
agg_data = all_data.groupby(['species', 'chemical_superclass']).size().reset_index(name='recurrence')

# Define a sequential color palette for 'chemical_superclass'
#color_palette = plotly.colors.sequential.Cividis

# Step 5: Create the stacked barplot with the Vivid color palette
fig = px.bar(agg_data, y=agg_data['species'].apply(lambda x: f"<i>{x}</i>"), x='recurrence',
             title='Stacked Barplot of Predicted Superclasses Recurrence for All Species',
             labels={'recurrence': 'Recurrence'},
             color='chemical_superclass',
             color_discrete_sequence=px.colors.qualitative.Vivid,  # Use the Vivid color palette
             #color_discrete_sequence=color_palette,
             category_orders={'chemical_superclass': sorted(agg_data['chemical_superclass'].unique())},
             orientation='h'#,
             #hover_data=['chemical_compound'] 
            )

# Modify the y-axis label
fig.update_yaxes(title_text='Species')

# Set a white background
fig.update_layout(plot_bgcolor='white')

# Modify the size of the figure
fig.update_layout(width=1500, height=900)

# Save the figure as an HTML file
fig.write_html("Wikidata_superclass_barplot.html")

# Show or save the figure
fig.show()


In [152]:
import pandas as pd
import plotly.express as px
import os

# Define the Vivid color palette (you can customize this as needed)
vivid_palette = px.colors.qualitative.Vivid

# Define custom colors for the 7 pathway categories
pathway_colors = {
    'Category1': 'red',
    'Category2': 'blue',
    'Category3': 'green',
    'Category4': 'purple',
    'Category5': 'orange',
    'Category6': 'pink',
    'Category7': 'brown'
}

# Step 1: Initialize an empty DataFrame to store all data
all_data = pd.DataFrame(columns=['queried_taxaLabel', 'predicted_pathway'])

# Step 2: Read data from all .tsv files in the output_folder
for filename in os.listdir(output_folder):
    if filename.endswith(".tsv"):
        file_data = pd.read_csv(os.path.join(output_folder, filename), sep='\t')
        all_data = pd.concat([all_data, file_data])

# Step 3: Rename the "queried_taxaLabel" column to "species"
all_data.rename(columns={'queried_taxaLabel': 'species'}, inplace=True)

# Remove 'API Error-API Error' and 'Not Classified-Not Classified'
all_data = all_data[(all_data['predicted_pathway'] != 'API Error') & (all_data['predicted_pathway'] != 'Not Classified')]

# Step 4: Group and aggregate data to calculate recurrence
agg_data = all_data.groupby(['species', 'predicted_pathway']).size().reset_index(name='recurrence')

# Step 5: Create the stacked barplot with custom colors
fig = px.bar(agg_data, y=agg_data['species'].apply(lambda x: f"<i>{x}</i>"), x='recurrence',
             title='Stacked Barplot of Predicted Pathways Recurrence for All Species',
             labels={'recurrence': 'Recurrence'},
             color='predicted_pathway',
             color_discrete_map=pathway_colors,  # Use custom colors
             category_orders={'predicted_pathway': sorted(agg_data['predicted_pathway'].unique())},
             orientation='h'
            )

# Modify the y-axis label
fig.update_yaxes(title_text='Species')

# Set a white background
fig.update_layout(plot_bgcolor='white')

# Modify the size of the figure
fig.update_layout(width=1500, height=900)

# Save the figure as an HTML file
fig.write_html("Wikidata_pathway_barplot.html")

# Show or save the figure
fig.show()
